In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/emergency_vs_non-emergency_dataset/emergency_train.csv
/kaggle/input/emergency_vs_non-emergency_dataset/images/890.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1139.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1946.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1824.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1190.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/2218.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/745.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/61.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1458.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1805.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1169.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/50.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1668.jpg
/kaggle/input/emergency_vs_non-emergency_dataset/images/1422.jpg
/kaggle/input/emergency_vs_

# The aim here is to classify the images of vehicles as emergency or non-emergency.

In [2]:
from tqdm import tqdm # tqdm is a progress bar library with good support for nested loops and jupyter notebooks

# for reading and displaying image
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model 
from sklearn.metrics import accuracy_score

# Pytorch libraries and modules 
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# torchvision for pre trained models
from torchvision import models

# Loading Dataset

In [4]:
# loading dataset

train = pd.read_csv("/kaggle/input/emergency_vs_non-emergency_dataset/emergency_train.csv")
train.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [5]:
# loading training images
train_img = []
for img_name in tqdm(train['image_names']):
    #print(img_name)
    
    # defining the image path
    image_path='/kaggle/input/emergency_vs_non-emergency_dataset/images/' + img_name
    #print(image_path)
    
    # reading the image
    img = imread(image_path)
    #print(img)
    
    # normalizing the pixel values 
    img = img/255
    #print(img)
    
    # resizing the image to (224,224,3)
    img = resize(img, output_shape=(224,224,3), mode='constant', anti_aliasing=True)
    #print(img)
    
    #coverting the type of pixel to float 32
    img = img.astype('float32')
    
    # appending the image into the list
    train_img.append(img)
    #print(train_img)
    
# converting the list to numpy array
train_x = np.array(train_img)
train_x.shape  

100%|██████████| 1646/1646 [00:19<00:00, 84.97it/s] 


(1646, 224, 224, 3)

# Creating training and validation set

In [6]:
# defining the target
train_y = train['emergency_or_not'].values

# create validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 13, stratify=train_y)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((1481, 224, 224, 3), (1481,)), ((165, 224, 224, 3), (165,)))

# Converting Images To Torch Format

In [7]:
# converting training images into torch format
train_x = train_x.reshape(1481, 3, 224, 224)
train_x  = torch.from_numpy(train_x)
print(type(train_x), train_x.size())

<class 'torch.Tensor'> torch.Size([1481, 3, 224, 224])


In [8]:

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)

# converting validation images into torch format
val_x = val_x.reshape(165, 3, 224, 224)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

# Defining the model architecture

In [9]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear_layers = Sequential(
            Linear(32 * 56 * 56, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

# Defining Model Parameters

In [10]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=2, bias=True)
  )
)


# Training The Model

In [11]:
torch.manual_seed(0)

# batch size of the model
batch_size = 128

# number of epochs to train the model
n_epochs = 25

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

 17%|█▋        | 2/12 [00:00<00:00, 12.03it/s]

epoch: 	 1 	 training loss: 	 0.6985050439834595


 17%|█▋        | 2/12 [00:00<00:00, 11.96it/s]

epoch: 	 2 	 training loss: 	 0.6390030135711035


 17%|█▋        | 2/12 [00:00<00:00, 12.08it/s]

...

epoch: 	 23 	 training loss: 	 0.39099737008412677


 17%|█▋        | 2/12 [00:00<00:00, 11.81it/s]

epoch: 	 24 	 training loss: 	 0.3820014223456383


100%|██████████| 12/12 [00:01<00:00, 11.13it/s]

epoch: 	 25 	 training loss: 	 0.3851035311818123


# Prediction on the training set 

In [12]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    print(">>>>>>>>>>", type(predictions), type(batch_y))
    batch_y = batch_y.cpu().numpy()
    print(">>>>", type(predictions), type(batch_y))
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

 17%|█▋        | 2/12 [00:00<00:00, 12.85it/s]

>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


 50%|█████     | 6/12 [00:00<00:00, 12.88it/s]

>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


 67%|██████▋   | 8/12 [00:00<00:00, 12.97it/s]

>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


100%|██████████| 12/12 [00:00<00:00, 13.40it/s]

>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
>>>>>>>>>> <class 'numpy.ndarray'> <class 'torch.Tensor'>
>>>> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
training accuracy: 	 0.8780768407534246


# Prediction on the validation set

In [13]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    batch_y=batch_y.cpu().numpy()
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|██████████| 2/2 [00:00<00:00, 20.46it/s]

validation accuracy: 	 0.6972128378378378


# Improving our model

# 1. Paucity of Data Available for Training our Model

### Data augmentation is the process of generating new data or increasing the data for training the model without actually collecting new data.

# 2. Model Overfitting 

### A model is said to overfit when it performs really well on the training set but the performance drops on the validation set (or unseen data).

# Now lets add a dropout layer to our architecture and check its performance.

# Model Architecture

In [14]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # dropout layer
            Dropout(),
            # Defining another 2D convolution layer
            Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # dropout layer
            Dropout(),
        )

        self.linear_layers = Sequential(
            Linear(32 * 56 * 56, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


# Model Parameters

In [15]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.5, inplace=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=2, bias=True)
  )
)


# Training the Model

In [16]:
torch.manual_seed(0)

# batch size of the model
batch_size = 128

# number of epochs to train the model
n_epochs = 25

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

 17%|█▋        | 2/12 [00:00<00:00, 11.61it/s]

epoch: 	 1 	 training loss: 	 0.7837458252906799


 17%|█▋        | 2/12 [00:00<00:00, 11.57it/s]

epoch: 	 2 	 training loss: 	 0.6634471565485001


 17%|█▋        | 2/12 [00:00<00:00, 11.52it/s]

...

epoch: 	 23 	 training loss: 	 0.5506446212530136


 17%|█▋        | 2/12 [00:00<00:00, 11.40it/s]

epoch: 	 24 	 training loss: 	 0.5441155781348547


100%|██████████| 12/12 [00:01<00:00, 10.95it/s]

epoch: 	 25 	 training loss: 	 0.5515835906068484


# Checking Model Performance

In [17]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    batch_y=batch_y.cpu().numpy()
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

100%|██████████| 12/12 [00:00<00:00, 13.25it/s]

training accuracy: 	 0.7447559931506849


# Similarly, let’s check the validation accuracy:

In [18]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    batch_y = batch_y.cpu().numpy()
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|██████████| 2/2 [00:00<00:00, 22.04it/s]

validation accuracy: 	 0.6911951013513513


Description              | Training Accuracy | Validation Accuracy  |
--------------           | -------------     | -------------        |
Without DropOut          |         87.80     |     69.72            |
With DropOut (p=0.5)     |         74.83     |     74.52            |

The table above shows accuracy without Dropout and with Dropout. 
<br/> In the first case the training accuracy and the validation accuracy are not in syncronization. The accuracy rapidly decreases on the validation set. <br/>
When we use drop both training and validation accuracies come in syncoronization.

# 3. Model UnderFitting
### To overcome underfitting we have the following solutions:
* 1. Increase the training data
* 2. Make a comples model
* 3. Increase the training epoch

### For our model we are not facing this problem of underfitting.

# 4. Training time is too high

Sometime the neural networks take a lot of time to converge. The main reason behind this is the change in the distribution of inputs to the layers of the neural network. 
* To overcome this problem, we can apply batch normalization wherein we normalize the activations of hidden layers and try to make the same distribution.

Now lets add batchnorm layers to the architecture and check how it performs for the vehicle classification problem.

In [19]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # batch normalization layer
            BatchNorm2d(16),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),    
            # batch normalization layer
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(32 * 56 * 56, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

# Defining Model Parameters

In [20]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.00005)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=2, bias=True)
  )
)


# Train the Model

In [21]:
torch.manual_seed(0)

# batch size of the model
batch_size = 128

# number of epochs to train the model
n_epochs = 5

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

 17%|█▋        | 2/12 [00:00<00:00, 10.16it/s]

epoch: 	 1 	 training loss: 	 0.9493299623330435


  0%|          | 0/12 [00:00<?, ?it/s]

epoch: 	 2 	 training loss: 	 0.5637829179565111


  0%|          | 0/12 [00:00<?, ?it/s]

epoch: 	 3 	 training loss: 	 0.3885498568415642


 17%|█▋        | 2/12 [00:00<00:00, 10.21it/s]

epoch: 	 4 	 training loss: 	 0.3035188727080822


100%|██████████| 12/12 [00:01<00:00,  9.17it/s]

epoch: 	 5 	 training loss: 	 0.25058118129769963


We can see the model is learnig very fast. It can be seen that training loss of 0.25 in the fifth loss itself when compared to the training loss of 0.38 in the 25th epoch in our earlier attempts.

# Prediction on the training set 

In [22]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    batch_y = batch_y.cpu().numpy()
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

100%|██████████| 12/12 [00:00<00:00, 12.32it/s]

training accuracy: 	 0.9534371432648402


# Prediction on the training set 

In [23]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    batch_y=batch_y.cpu().numpy()
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|██████████| 2/2 [00:00<00:00, 19.26it/s]

validation accuracy: 	 0.6584670608108107


As we can see adding batch normalization reduced the traing time but we have an issue. The model is now overfitting. We got an accuracy of 94% on training and 63% on validation.<br/>
We have not added the dropout layer to this model.

# Now, lets use all the techniques we have learnt so far to improve our model

# Model Architecture

In [24]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(16),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(),
            # Defining another 2D convolution layer
            Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(),
        )

        self.linear_layers = Sequential(
            Linear(32 * 56 * 56, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

# Defining Model Parameters

In [25]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.00025)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.5, inplace=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=2, bias=True)
  )
)


# Train The Model

In [26]:
torch.manual_seed(0)

# batch size of the model
batch_size = 128

# number of epochs to train the model
n_epochs = 10

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

 17%|█▋        | 2/12 [00:00<00:00, 10.08it/s]

epoch: 	 1 	 training loss: 	 4.997635851303737


 17%|█▋        | 2/12 [00:00<00:00, 10.11it/s]

epoch: 	 2 	 training loss: 	 1.8881468524535496


  0%|          | 0/12 [00:00<?, ?it/s]

...

epoch: 	 8 	 training loss: 	 0.3156622275710106


  0%|          | 0/12 [00:00<?, ?it/s]

epoch: 	 9 	 training loss: 	 0.27080752700567245


100%|██████████| 12/12 [00:01<00:00,  8.98it/s]

epoch: 	 10 	 training loss: 	 0.31075844417015713


# Prediction on the training set 

In [27]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    batch_y=batch_y.cpu().numpy()
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

100%|██████████| 12/12 [00:00<00:00, 12.04it/s]

training accuracy: 	 0.8954855165525114


# Prediction on the validation set 

In [28]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    batch_y=batch_y.cpu().numpy()
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|██████████| 2/2 [00:00<00:00, 19.31it/s]

validation accuracy: 	 0.6389358108108107


# We can improve the model as follows:
* 1. Tune the dropout rate
* 2. Add or reduce the number of convolutional layers.
* 3. Add or reduce the number of dense layer.
* 4. Tune the number of neurons in hidden layers, etc. 

# Thank You